In [ ]:
import cv2
import mediapipe as mp

# Inicializa MediaPipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Configura la cámara
cap = cv2.VideoCapture(0)

# Verifica si la cámara está disponible
if not cap.isOpened():
    print("Error: No se puede acceder a la cámara.")
    exit()

# Inicia el modelo de detección de pose
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("No se pudo leer el cuadro de la cámara.")
            break

        
        # Convierte la imagen de BGR a RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Procesa la imagen para detectar poses
        results = pose.process(rgb_frame)

        # Dibuja las anotaciones de pose en la imagen original
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
            )

        # Muestra la imagen procesada
        cv2.imshow('Skeleton Detection', frame)

        # Salir con la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Libera los recursos
cap.release()
cv2.destroyAllWindows()


In [4]:
import cv2
import mediapipe as mp
import time

# Inicializa los módulos de Mediapipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Configura la cámara y el video
cap = cv2.VideoCapture(0)  # Cámara en vivo
video_cap = cv2.VideoCapture('dance_video.mp4')  # Video del archivo

# Variable para medir el tiempo
start_time = time.time()

# Inicia el modelo de detección de pose
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        # Captura cuadro del video
        ret_vid, frame_vid = video_cap.read()
        if not ret_vid:
            print("Se alcanzó el final del video.")
            video_cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reinicia el video
            ret_vid, frame_vid = video_cap.read()

        frame_vid = cv2.resize(frame_vid, None, fx=0.5, fy=0.5)
        # Procesa el video
        rgb_frame_vid = cv2.cvtColor(frame_vid, cv2.COLOR_BGR2RGB)
        results_vid = pose.process(rgb_frame_vid)

        if results_vid.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame_vid, results_vid.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=2)
            )

        # Captura los puntos cada 30 segundos
        current_time = time.time()
        if current_time - start_time >= 30:

            # Imprime los puntos del video en la terminal
            if results_vid.pose_landmarks:
                print("Puntos detectados (Video):")
                for id, landmark in enumerate(results_vid.pose_landmarks.landmark):
                    print(f"Punto {id}: (x: {landmark.x}, y: {landmark.y}, z: {landmark.z}, visibilidad: {landmark.visibility})")
                print("\n---\n")

            # Reinicia el contador de tiempo
            start_time = current_time

        # Muestra las imágenes procesadas
        cv2.imshow('Skeleton Detection (Video)', frame_vid)

        # Termina el bucle si se presiona la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Libera los recursos
cap.release()
video_cap.release()
cv2.destroyAllWindows()

In [12]:
import cv2
import mediapipe as mp
import numpy as np
import time
import json

# Inicializa los módulos de Mediapipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Configuración del video de referencia
dance_video = 'dance_video.mp4'

# Función para extraer puntos de referencia y guardarlos en un archivo JSON
def extraer_y_guardar_puntos(video_path, output_file):
    puntos_video_referencia = []

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        video_cap = cv2.VideoCapture(video_path)
        while True:
            ret_vid, frame_vid = video_cap.read()
            if not ret_vid:
                break

            rgb_frame_vid = cv2.cvtColor(frame_vid, cv2.COLOR_BGR2RGB)
            results_vid = pose.process(rgb_frame_vid)

            if results_vid.pose_landmarks:
                puntos = []
                for landmark in results_vid.pose_landmarks.landmark:
                    puntos.append({
                        "x": landmark.x,
                        "y": landmark.y,
                        "z": landmark.z,
                        "visibility": landmark.visibility
                    })
                puntos_video_referencia.append(puntos)

        video_cap.release()

    # Guarda los puntos en un archivo JSON
    with open(output_file, 'w') as f:
        json.dump(puntos_video_referencia, f)

    print(f"Puntos de referencia guardados en {output_file}")

# Función para calcular la distancia relativa entre puntos clave
def calcular_distancia_escala(puntos):
    """
    Calcula una medida de escala basada en la distancia entre los hombros
    (puntos 11 y 12 en Mediapipe Pose).
    """
    hombro_izquierdo = puntos[11]
    hombro_derecho = puntos[12]
    return np.linalg.norm(np.array([hombro_izquierdo["x"], hombro_izquierdo["y"]]) -
                          np.array([hombro_derecho["x"], hombro_derecho["y"]]))

# Código para jugar comparando los puntos guardados

def jugar_con_puntos(json_file, video_path):
    # Carga los puntos del archivo JSON
    with open(json_file, 'r') as f:
        puntos_video_referencia = json.load(f)

    cap = cv2.VideoCapture(0)  # Cámara en vivo
    video_cap = cv2.VideoCapture(video_path)  # Video de referencia

    def calcular_similitud(puntos1, puntos2, escala1, escala2):
        """
        Calcula la similitud entre dos poses ajustando las coordenadas por sus escalas relativas.
        """
        if not puntos1 or not puntos2 or len(puntos1) != len(puntos2):
            return 0

        distancias = []
        for p1, p2 in zip(puntos1, puntos2):
            p1_escalado = np.array([p1["x"] / escala1, p1["y"] / escala1, p1["z"] / escala1])
            p2_escalado = np.array([p2["x"] / escala2, p2["y"] / escala2, p2["z"] / escala2])
            distancias.append(np.linalg.norm(p1_escalado - p2_escalado))

        return np.mean(distancias)

    frame_idx = 0

    while True:
        # Leer cuadro del video de referencia
        ret_vid, frame_vid = video_cap.read()
        if not ret_vid:
            video_cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reinicia el video
            frame_idx = 0
            ret_vid, frame_vid = video_cap.read()

        if frame_idx < len(puntos_video_referencia):
            puntos_referencia = puntos_video_referencia[frame_idx]

            # Dibujar puntos y conexiones en el video de referencia
            altura, ancho, _ = frame_vid.shape
            for conexion in mp_pose.POSE_CONNECTIONS:
                inicio = puntos_referencia[conexion[0]]
                fin = puntos_referencia[conexion[1]]
                inicio_px = (int(inicio["x"] * ancho), int(inicio["y"] * altura))
                fin_px = (int(fin["x"] * ancho), int(fin["y"] * altura))
                cv2.line(frame_vid, inicio_px, fin_px, (255, 0, 0), 2)

            for punto in puntos_referencia:
                px = (int(punto["x"] * ancho), int(punto["y"] * altura))
                cv2.circle(frame_vid, px, 5, (0, 255, 0), -1)

        # Leer cuadro de la cámara en vivo
        ret_cam, frame_cam = cap.read()
        if not ret_cam:
            print("No se puede acceder a la cámara.")
            break
        
        frame_cam = cv2.resize(frame_cam, None, fx=0.5, fy=0.5)
        rgb_frame_cam = cv2.cvtColor(frame_cam, cv2.COLOR_BGR2RGB)
        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            results_cam = pose.process(rgb_frame_cam)

            if results_cam.pose_landmarks:
                puntos_usuario = [
                    {"x": lm.x, "y": lm.y, "z": lm.z, "visibility": lm.visibility}
                    for lm in results_cam.pose_landmarks.landmark
                ]

                if frame_idx < len(puntos_video_referencia):
                    escala_referencia = calcular_distancia_escala(puntos_referencia)
                    escala_usuario = calcular_distancia_escala(puntos_usuario)

                    similitud = calcular_similitud(puntos_referencia, puntos_usuario, escala_referencia, escala_usuario)

                    if similitud < 0.1:
                        puntuacion = 100
                    elif similitud < 0.2:
                        puntuacion = 50
                    else:
                        puntuacion = 0

                    cv2.putText(frame_cam, f'Puntuacion: {puntuacion}', (10, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            if results_cam.pose_landmarks:
                mp_drawing.draw_landmarks(
                    frame_cam, results_cam.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=2, circle_radius=2)
                )

        # Mostrar ambos cuadros
        combinado = np.hstack((cv2.resize(frame_vid, (640, 480)), cv2.resize(frame_cam, (640, 480))))
        cv2.imshow('Comparacion: Video vs Usuario', combinado)

        frame_idx += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    video_cap.release()
    cv2.destroyAllWindows()

# Ejemplo de uso
if __name__ == "__main__":
    modo = input("Seleccione modo: 'extraer' para guardar puntos o 'jugar' para comparar: ")

    if modo == 'extraer':
        extraer_y_guardar_puntos(dance_video, 'puntos_referencia.json')
    elif modo == 'jugar':
        jugar_con_puntos('puntos_referencia.json', dance_video)
